In [2]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
# Requests
import requests
from time import sleep
from tqdm import tqdm

# Para convertir de string a diccionario
import ast

import sys
sys.path.append("../")
import src.soporte_spotify as api

In [3]:
sp = api.load_credentials()
sp.current_user()

{'country': 'ES',
 'display_name': 'MrApple',
 'email': 'ruiperezgonzalo4@gmail.com',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'external_urls': {'spotify': 'https://open.spotify.com/user/gonzaloruiperez'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/users/gonzaloruiperez',
 'id': 'gonzaloruiperez',
 'images': [],
 'product': 'premium',
 'type': 'user',
 'uri': 'spotify:user:gonzaloruiperez'}

# Verificar si nos han bloqueado
- Tenemos un limite diario y si no spotify devuelve 429, algo que con spotipy no se ve, asi que usamos requests

In [4]:
token = api.request_token()
playlist_id = "1IMxg7e2dwhawHaoBfWWIH" 
url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

headers = {
    "Authorization": f"Bearer {token}"
}

response = requests.get(url, headers=headers)


print(f"{response.status_code}, Detalles: {response.text}")
print(response.headers)


Token obtenido con éxito
429, Detalles: Too many requests
{'cache-control': 'private, max-age=0', 'retry-after': '77300', 'access-control-allow-origin': '*', 'access-control-allow-headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token', 'access-control-allow-methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'access-control-allow-credentials': 'true', 'access-control-max-age': '604800', 'content-encoding': 'gzip', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff', 'date': 'Fri, 03 Jan 2025 15:34:07 GMT', 'server': 'envoy', 'Via': 'HTTP/2 edgeproxy, 1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}


In [5]:
zara = pd.read_csv('../datos/01 Spotify/01_followers_zara.csv',index_col=0)
primark = pd.read_csv("../datos/01 Spotify/02_followers_primark.csv",index_col=0)
nike = pd.read_csv("../datos/01 Spotify/03_followers_nike.csv",index_col=0)
adidas = pd.read_csv("../datos/01 Spotify/04_followers_adidas.csv",index_col=0)
hm = pd.read_csv("../datos/01 Spotify/05_followers_hm.csv",index_col=0)
pullbear = pd.read_csv("../datos/01 Spotify/06_followers_pull_and_bear.csv",index_col=0)
bershka = pd.read_csv("../datos/01 Spotify/07_followers_bershka.csv",index_col=0)
stradivarius = pd.read_csv("../datos/01 Spotify/08_followers_stradivarius.csv",index_col=0)
mango = pd.read_csv("../datos/01 Spotify/09_followers_mango.csv",index_col=0)
desigual = pd.read_csv("../datos/01 Spotify/10_followers_desigual.csv",index_col=0)

# Zara
### Obtener ID canciones
- Por cada id de playlist, sacaremos los tracks con su respectivo id de canción


In [6]:
zara.sample()

,brand,username,user_id,playlists
501,Zara,maizpuro,maizpuro,"{'Mara SOUL ': '4jxi1tC1b0Q0JEVll9CyvH', 'Luna..."


### Convertir columna playlists a diccionario
- utilizamos la librería ast para obtener el diccionario

In [7]:
zara["playlists"] = zara["playlists"].apply(ast.literal_eval)
type(zara["playlists"][0])

dict

### Sacamos a una lista todos los ids de playlists

In [8]:
playlist_ids = []
for ids in zara["playlists"]:
    playlist_ids.append(list(ids.values()))

zara["playlist_ids"] = playlist_ids

### Obtener los artistas de cada usuario

In [1]:
lista_artistas = []
for id_playlist in tqdm(zara["playlist_ids"], desc="Obteniendo artistas"):
   artistas =  api.obtener_artistas(sp, id_playlist)
   lista_artistas.append(artistas)

zara["artistas"] = lista_artistas
zara.to_csv("../datos/01 Spotify/00_GuardadoZara.csv")

Obteniendo artistas:   0%|          | 0/873 [00:00<?, ?it/s]

Error 429: Rate limit alcanzado. Esperando 77300 segundos...
